# Survey Data on Broadband Access for Public Libraries

This notebook explores and cleans the data from the survey on broadband access for public libraries. The survey data is used along with the geocoded data from the libraries (`geolib_df`) obtained from `utils/geocoding_libs.py`. The resulting dataset from this process is a library dataset that contains the name, address and geolocation of the libraries, as well as data on broadband access (`data/lib_data_plot.csv`).

There survey has several questions, but we'll focus in the ones that ask about broadband access and upload/download speeds. After cleaning the data, we'll merge both datasets keeping the `geolib_df` as a reference, as it has the universe of libraries in Illinois and the survey is a subset of those.

## I. Loading Libraries and Data

In [2]:
import sys
sys.path.append('../')
import pandas as pd

from utils.geocoding_libs import geocode_lib

In [3]:
survey_df = pd.read_excel("../data/public_library_survey.xlsx",
                          usecols=[8,10,17,18,21,22,23,24,25,26],
                          engine="openpyxl", skiprows=[1,2])

survey_df.rename(columns={8: "lib_name", 10: "lib_address", 17: "broadband",
                  18: "wifi", 21: "down_open", 22: "upload_open",
                  23: "down_midday", 24: "upload_midday",
                  25: "down_close", 26: "upload_close"}, inplace=True)

geolib_df = pd.read_csv("../data/geocoded_lib_data_public.csv")

## II. Cleaning Survey Data

We make sure that we only keep information that has data for all rows, so we begin by removing `NAs`. This gives us a list of the 592 libraries that at least started the survey:

In [4]:
survey_df.dropna(how="all", inplace=True)
survey_df.shape

(592, 10)

In some cases, the survey is filled by more than one employee in the same library, so we remove those duplicates as we only want to keep one register per library:

In [5]:
# Duplicates
survey_df[survey_df.duplicated()].head()

,lib_name,lib_address,broadband,wifi,down_open,upload_open,down_midday,upload_midday,down_close,upload_close
98,Belleville Public Library,121 E. Washington St.,Yes,Yes,86.0,85.0,85.0,84.0,88.0,86.0
118,Cicero Public Library,"5225 W. Cermak Rd., Cicero, IL, 60804",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,Harvey Public Library District,15441 Turlington Ave,Yes,Yes,320.0,320.0,320.0,320.0,320.0,320.0
203,Oak Park Public Library,834 Lake St,Yes,Yes,236.0,241.0,169.0,109.0,243.0,236.0
216,Alpha Park Public Library District,"3527 S Airport Rd. Bartonville, IL 616107",Yes,Yes,100.0,100.0,100.0,100.0,100.0,100.0


In [6]:
# Drop all-row duplicates
survey_df.drop_duplicates(inplace=True)
survey_df.shape

(577, 10)

There is also more than one entry per library with different values in some columns like address (*e.g.*, Rockford Public Library) or in data columns (*e.g.,* Oak Park Public Library - 834 Lake St):

In [7]:
# Same name, different values for address:
survey_df[survey_df.loc[:,"lib_name"] == "Rockford Public Library"]

,lib_name,lib_address,broadband,wifi,down_open,upload_open,down_midday,upload_midday,down_close,upload_close
509,Rockford Public Library,6685 East State Street,Yes,Yes,320.0,320.0,320.0,320.0,320.0,320.0
510,Rockford Public Library,1238 S. Winnebago Street,Yes,Yes,320.0,320.0,320.0,320.0,320.0,320.0
512,Rockford Public Library,214 North Church Street,Yes,Yes,320.0,320.0,320.0,320.0,320.0,320.0


In [8]:
# Same name and address, different value for variables:
survey_df[survey_df.loc[:,"lib_address"] == "834 Lake St"]

,lib_name,lib_address,broadband,wifi,down_open,upload_open,down_midday,upload_midday,down_close,upload_close
144,Oak Park Public Library,834 Lake St,Yes,Yes,236.0,241.0,169.0,109.0,243.0,236.0
479,Oak Park Public Library,834 Lake St,Yes,Yes,245.0,237.0,186.0,183.0,239.0,237.0


We remove those duplicates by subsetting the pattern to `lib_name`. Before that, we remove trailing whitespaces since library names (`lib_name`) and addresses (`lib_address`) are not homogeneous:

In [9]:
survey_df.loc[:,"lib_name"] = survey_df.loc[:,"lib_name"].astype('str').apply(lambda x: x.strip())
survey_df.loc[:,"lib_address"] = survey_df.loc[:,"lib_address"].astype('str').apply(lambda x: x.strip())

In [10]:
survey_df.drop_duplicates(subset="lib_name",inplace=True)
survey_df.shape

(450, 10)

And finally, we perfom a `value_counts()` operation over the `lib_address` to check that there are no more duplicates:

In [11]:
survey_df.loc[:,"lib_address"].value_counts().to_frame().reset_index()

,lib_address,count
0,"201 W. Adams St., Havana, IL 62644",1
1,1270 Mulholland Street,1
2,210 W Green St,1
3,"10 South Batavia Avenue, Batavia IL 60510",1
4,"307 South Madison, Morrison, IL 61270",1
...,...,...
445,"1501 Ellinwood St, Des Plaines, 60016",1
446,207 N Walnut,1
447,"600 S. Main St., PO Box 211, Buffalo, IL 62515",1
448,"616 Hiawatha Drive, Carol Stream, 60188",1


## III. Geocoding the Survey Data

We create a dictionary with `lib_name` and `lib_address` to use the geocoding API:

In [12]:
# Convert the DataFrame to a dictionary
library_dict = survey_df.set_index('lib_name')['lib_address'].to_dict()

# Convert each value in the dictionary to a list
library_dict = {key: [value] for key, value in library_dict.items()}

# Call function to create dataframe with location
library_df = geocode_lib(library_dict)
library_df.head(3)

,lib_name,lib_address,latitude,longitude
0,Havana Public Library District,"201 W. Adams St., Havana, IL 62644",40.299153,-90.062597
1,Riverton Village library,"1200 Riverton Road, Riverton, Illionis, 62561",39.859524,-89.531471
2,Bethalto Public Library District,"321 S. Prairie St., Bethalto, IL 62010",38.907462,-90.040347


From the 450 records after initial cleaning, 370 had a valid address to retrieve coordinates:

In [13]:
library_df.shape

(371, 4)

## IV. Data Merging 

We'll merge both the original `geolib_df` and the geocoded survey df using `lib_name` as a key. We'll preserve all the observations from the original dataset since it contains the universe of libraries in Illinois while the survey data is a subset.

In order to do this we first need to add the location data to the survey data. This will be represented in a dataset called `geosurvey_df`:

In [14]:
# Create new dataframe by merging the survey_df and the geocoded one
geosurvey_df = pd.merge(library_df,survey_df, how="left", on="lib_name")
geosurvey_df.rename(columns={"lib_name": "lib_name_survey",
                             "lib_address_x":"lib_address_survey"},
                             inplace=True)
geosurvey_df.drop("lib_address_y", axis=1, inplace=True)

In [15]:
geosurvey_df.head(3)

,lib_name_survey,lib_address_survey,latitude,longitude,broadband,wifi,down_open,upload_open,down_midday,upload_midday,down_close,upload_close
0,Havana Public Library District,"201 W. Adams St., Havana, IL 62644",40.299153,-90.062597,Yes,Yes,320.0,198.0,320.0,320.0,320.0,320.0
1,Riverton Village library,"1200 Riverton Road, Riverton, Illionis, 62561",39.859524,-89.531471,Yes,Yes,93.0,94.0,93.0,94.0,94.0,94.0
2,Bethalto Public Library District,"321 S. Prairie St., Bethalto, IL 62010",38.907462,-90.040347,Yes,Yes,203.0,7.0,146.0,3.0,209.0,7.0


In [16]:
geosurvey_df.shape

(371, 12)

And now, we create the `merged_df` as described before:

In [17]:
# Add data where there is info Keep old dataframe with new survey data 
merged_df = pd.merge(geolib_df,
                     geosurvey_df,
                     how="left",
                     on=["latitude","longitude"])

As a final check, we compare the dimensions of `geolib_df` and `merged_df` to check that there are no duplicates:

In [18]:
print("Number of rows in the geolib_df:",geolib_df.shape[0])
print("Number of rows in the merged_df:",merged_df.shape[0])

Number of rows in the geolib_df: 781
Number of rows in the merged_df: 781


In [20]:
merged_df.head(2)

,lib_name,lib_address,latitude,longitude,lib_name_survey,lib_address_survey,broadband,wifi,down_open,upload_open,down_midday,upload_midday,down_close,upload_close
0,A. Herr Smith & E.E. Smith Loda Township Library,"105 East Adams Street Loda, IL 60948-0247",40.517828,-88.070665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acorn Public Library District,"15624 South Central Avenue Oak Forest, IL 6045...",41.607358,-87.756728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## V. Exporting Data

We create average download and upload sppeds before exporting the data:

In [24]:
merged_df.loc[:,"avg_down_speed"] = merged_df.loc[:,["down_open","down_midday","down_close"]].mean(axis=1)
merged_df.loc[:,"avg_up_speed"] = merged_df.loc[:,["upload_open","upload_midday","upload_close"]].mean(axis=1)

And finally we export the dataset, which contains survey data for 370 out of 781 libraries (approx. 50%):

In [106]:
merged_df.to_csv("../data/lib_data_plot.csv", index=False)